### HELLO! This is a simple tool I have created to query optical photometry for nearby galaxies at a location of you choosing from SDSS DR12 in Vizier. Hope you find it useful! --Vic 07/31/2020

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import IPython.display
import ipywidgets as widgets
import pickle
import dustmaps.sfd
import numpy as np
import os
from astropy import units as u

# This file contains all external dependencies
from utilities import util

dustmaps.sfd.fetch()
sfd = dustmaps.sfd.SFDQuery()

Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.
Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.


# Name of GRB, metadata, coordinates, extinction

In [2]:
data_path = 'data/'
object_name = 'GRB201015A'
ra  = '23:37:16.46'
dec = '53:24:57.7'

# Parse coord
coord = util.parse_coord(ra, dec)

# Get Swift data
swift_data = util.get_grb_data(object_name)

# Get MW Av
av = sfd(coord) * 3.1
print('Milky Way Av={0} mag'.format(('%7.4f'%av).strip()))

Successfully downloaded Swift GRB data...
Got GRB metadata
Time: 22:50:13
Trigger Number: 1000452
BAT T90: 9.78
XRT RA: 23:37:16.46
XRT Dec: 53:24:57.7
Milky Way Av=1.0500 mag


### Catalog search - this will grab metadata for sources around input coord

In [4]:
# This method will search and crossmatch catalogs given an input list
# of catalogs.  The output is an astropy table keyed by as catname_key
# where key is the original key in each catalog
catnames = ['strm','ps1dr2','sdssdr12','2mass','unwise','glade','des']
catalog = util.search_catalogs(coord, catnames, search_radius=2.564*u.arcmin,
    match_radius=2.0*u.arcsec, outfile=os.path.join(data_path,object_name+'.cat'))
print(catalog['ra','dec'])

Searching strm catalog...
40 records in strm catalog
40 total records in catalog
Searching ps1dr2 catalog...
2298 records in ps1dr2 catalog
2298 total records in catalog
Searching sdssdr12 catalog...


TypeError: object of type 'NoneType' has no len()

### If you want an image of where you are looking at, you can use this!

In [13]:
# Here is an image of the cut out. I bet it looks super pretty!
# First tries SDSS, then PS1, then DES, then SkyMapper to get jpeg
# should have 100% sky coverage from those surveys
from astropy.coordinates import SkyCoord
coord = SkyCoord('12:00:00','10:00:00', unit=(u.hour,u.deg))
imfile = os.path.join(data_path,object_name+'_SDSS_cutout.jpg')
imname = util.getSDSS(coord, impix=1024, imsize=12*u.arcmin, outfile=imfile)

%matplotlib widget
fig, ax = plt.subplots(figsize=(10,10))
image = mpimg.imread(imfile)
ax.imshow(image)

plt.title(object_name,fontsize=20)
ax.hlines(512,532,572,color='limegreen',alpha=0.8,linewidth=2)
ax.vlines(512,532,572,color='limegreen',alpha=0.8,linewidth=2)

ax.hlines(1000,50,100,color='white',linewidth=3)
ax.vlines(100,950,1000,color='white',linewidth=3)
ax.text(95,945,'N',color='white',fontsize=15)
ax.text(35,1005,'E',color='white',fontsize=15)
    
# if XRT:
    # x_xrt, y_xrt = xrt_ra, xrt_dec
    # add_patch(patches.Circle((x_xrt,y_xrt),radius=xrt_radius,edgecolor='g',alpha=0.5,facecolor='none',linewidth=2,linestyle='--',label='XRT position at RA = %.3f, Dec = %.3f'%(xrt_ra, xrt_dec),picker=True))

#with open(object_name+'.txt', 'rb') as fp:
#    result = pickle.load(fp)
#for line in result:
#    x, y = 512+(coord.ra.deg-line[1])*3600/scale, 512+(coord.dec.deg-line[2])*3600/scale
#    ax.add_patch(patches.Circle((x,y),radius=2,edgecolor='r',alpha=0.5,facecolor='none',linewidth=2,linestyle='-',label='SDSS galaxy at RA = %.3f, Dec = %.3f'%(line[1],line[2]),picker=True))

fig.canvas.mpl_connect('click_pos', lambda event: onclick())
fig.canvas.mpl_connect('pick_event', lambda event: onpick())

TypeError: getSDSS() got an unexpected keyword argument 'outfile'

### Once you have the results, there are several things you can do. One of them is finding the probability chance coincident (Pcc) of the nearby galaxies which essentially tells you how likely that the GRB originated from a galaxy. The lower the Pcc, the more probable. 

In [ ]:
# The function you can use to calcilate that is here:
def pcc(r,m): 
    sigma = (1/(0.33*np.log(10)))*10**(0.33*(m-24)-2.44) 
    prob = 1-np.exp(-(np.pi*(r**2)*sigma)) 
    return prob

# An example of what a plot of Pcc as a function of distance looks like is here. This is what I did for GRB200522A. 
import numpy as np
with open(object_name+'.txt', 'rb') as fp:
    result = pickle.load(fp)

# Make a list of r_mag that corresponds to each galaxy near the GRB
r_mag = [line[7] for line in result]
#print(r_mag)

# Distance is in arcsecond, not arcminute of the galaxy from the GRB
distance = [line[0]*60 for line in result]
#print(distance)

# Make a list of photometric redshift
phot = [line[10] for line in result]
#print(phot)
    
prob = []
for i in range(len(r_mag)):
    prob.append(pcc(distance[i],r_mag[i]))
        
#print(prob)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]
# = plt.subplot(21.137, 0.316, c=0.5537)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.linewidth'] = 2
plt.scatter(distance, prob, marker = "o", c=phot, cmap='plasma', alpha = 0.6, s=195, edgecolors='black')
plt.ylabel('P$_{cc}$', fontsize = 20)
plt.xlabel('Distance (arcsec)', fontsize = 20)
clb = plt.colorbar()
clb.solids.set_alpha(1)
clb.set_label('phot-z', fontsize = 20, labelpad=15, rotation=90)
plt.savefig(object_name+'_pcc.png', format = 'pdf', dpi=300, bbox_inches="tight")
plt.show()

#### You can also sort out other lists of information such as the redshift and distance to make a scatter plot of redshifts as a function of distance. That can be helpful for figuring out whether the host is in a galaxy cluster or not. 